## Astrometry.net solutions - 15-Jul-25

Craig Lage - Attempting to find a.net solutions for crowded fields of 3I

In [ ]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.latiss import AUXTEL_LOCATION
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION
from lsst.geom import Point2D, SpherePoint, Angle, AngleUnit
from astropy.table import Table
from astroquery.astrometry_net import AstrometryNet
from lsst.summit.utils.astrometry.utils import runCharactierizeImage, filterSourceCatOnBrightest
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask, CharacterizeImageConfig
from lsst.summit.utils.plotting import plot
from lsst.daf.butler import Butler

In [ ]:
# Code originally from Merlin Fisher-Levine to solve StarTracker images
def runCharacterizeImage(exp, thresh, minPix):
    """Run the image characterization task, finding only bright sources.

    Parameters
    ----------
    exp : `lsst.afw.image.Exposure`
        The exposure to characterize.
    snr : `float`
        The SNR threshold for detection.
        Here we have modified it to be a numeric threshold
    minPix : `int`
        The minimum number of pixels to count as a source.

    Returns
    -------
    result : `lsst.pipe.base.Struct`
        The result from the image characterization task.
    """
    
    charConfig = CharacterizeImageConfig()
    charConfig.doMeasurePsf = False
    charConfig.doApCorr = True
    charConfig.doDeblend = True
    charConfig.doNormalizedCalibration = False
    charConfig.repair.doCosmicRay = False

    charConfig.detection.minPixels = minPix
    charConfig.detection.thresholdValue = thresh
    charConfig.detection.thresholdType = 'value'
    charConfig.detection.includeThresholdMultiplier = 1
    charConfig.detection.nSigmaToGrow = 0

    charConfig.psfIterations = 1
    charConfig.installSimplePsf.fwhm = 5
    charConfig.installSimplePsf.width = 51
    
    # fit background with the most simple thing possible as we don't need
    # much sophistication here. weighting=False is *required* for very
    # large binSizes.
    charConfig.background.approxOrderX = 1
    charConfig.background.approxOrderY = -1
    charConfig.background.binSize = max(exp.getWidth(), exp.getHeight())
    charConfig.background.weighting = False

    # set this to use all the same minimal settings as those above
    charConfig.detection.background = charConfig.background

    charTask = CharacterizeImageTask(config=charConfig)

    charResult = charTask.run(exp)
    return charResult



In [ ]:
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation',
                                              'LSSTCam/runs/nightlyValidation/20250425/w_2025_17/DM-50157'])
instrument = 'LSSTCam'

In [ ]:
expId = 2025071200566
detector = 30
ref_calexp = butler.get('preliminary_visit_image', detector=detector, visit=expId)
raw = butler.get('raw', detector=detector, exposure=expId)
mData=raw.getMetadata()
raw_wcs = raw.getWcs()
calexp_wcs = ref_calexp.getWcs()

In [ ]:
charResult = runCharacterizeImage(ref_calexp, 10000, 200)

In [ ]:
# Cut down to the brightest sources and eliminate close pairs
sourceCatalog = charResult.sourceCat
maxFlux = np.nanmax(sourceCatalog['base_CircularApertureFlux_3_0_instFlux'])
selectBrightestSource = sourceCatalog['base_CircularApertureFlux_3_0_instFlux'] > maxFlux * 0.99
brightestSource = sourceCatalog.subset(selectBrightestSource)
brightestCentroid = (brightestSource['base_SdssCentroid_x'][0], \
                     brightestSource['base_SdssCentroid_y'][0])
brightCatalog = sourceCatalog.subset(sourceCatalog['base_CircularApertureFlux_3_0_instFlux'] > maxFlux * 0.1)
print(f"expId:{expId}. Found {len(sourceCatalog)} sources, {len(brightCatalog)} bright sources")
print(f"Brightest centroid at {brightestCentroid}")
cat = brightCatalog
print(len(cat))
sources = sourceCatalog.asAstropy()
sources.keep_columns(['base_SdssCentroid_x', 'base_SdssCentroid_y', 'base_CircularApertureFlux_3_0_instFlux'])
print(len(sources))

# Eliminate close pairs
close_limit = 50 # pixels
for source1 in sources:
    x1 = source1['base_SdssCentroid_x']
    y1 = source1['base_SdssCentroid_y']
    for source2 in sources:
        x2 = source2['base_SdssCentroid_x']
        y2 = source2['base_SdssCentroid_y']
        if np.sqrt((x1-x2)**2 + (y1-y2)**2) < close_limit:
            sources.remove_rows([source2.index])
          
print(len(sources))            

In [ ]:
# Plot the sources on the image
fig = plt.figure(figsize=(16,16))

myPlot = plot(ref_calexp, stretch='ccs', figure=fig)
ax = myPlot.get_axes()[0]
ax.scatter(sources['base_SdssCentroid_x'],sources['base_SdssCentroid_y']\
            ,color='green', marker='o', s=150, facecolors='none')

ax.set_title(f"{expId} detector {detector}")
saveFilename = f"/home/c/cslage/u/LSSTCam/data/Astrometry-net_{expId}_{detector}.png"
plt.savefig(saveFilename)

In [ ]:
# Get astrometry.net solver
ast = AstrometryNet()
ast.api_key = 'xxawwhvleirxcswx'

In [ ]:
image_width = 4072
image_height = 4000
scale_units = 'arcsecperpix'
scale_type='ev' # ev means submit estimate and % error
scale_est = 0.20
scale_err = 2.0
center_ra = raw_wcs.pixelToSky(Point2D(2036, 2000)).getRa().asDegrees()
center_dec = raw_wcs.pixelToSky(Point2D(2036, 2000)).getDec().asDegrees()
radius = 0.5
wcs_header = ast.solve_from_source_list(sources['base_SdssCentroid_x'], sources['base_SdssCentroid_y'],
                                        image_width, image_height, scale_units=scale_units,
                                        scale_type=scale_type, scale_est=scale_est, scale_err=scale_err,
                                        center_ra=center_ra, center_dec=center_dec, radius=radius,
                                        solve_timeout=240, crpix_center=True)
print(wcs_header['CRVAL1'], wcs_header['CRVAL2'])

In [ ]:
# From the calexp_wcs
print(center_ra, center_dec)
print(mData['RA'], mData['DEC'])

In [ ]:
# From GAIA  265.3372772341, -18.3997811034  Object Gaia DR3 4123226195486361728

In [ ]:
# The astrometry.net solution.
wcs_header

In [ ]:
import astropy.wcs
from astropy.io import fits

wcs_from_fits = astropy.wcs.WCS(wcs_header)

In [ ]:
skyLocation = SkyCoord(265.178541*u.deg, -18.4558982*u.deg)
wcs_from_fits.world_to_pixel(skyLocation)

In [ ]:
raw_wcs.getPixelOrigin()

In [ ]:
raw_wcs.pixelToSky(Point2D(2036, 2000)).getRa().asDegrees()

In [ ]:
from lsst.afw.geom import SkyWcs


In [ ]:
test = SkyWcs(wcs_from_fits)

In [ ]:
expId = 2025071200536
detector = 94
calexp = butler.get('preliminary_visit_image', detector=detector, visit=expId)
raw = butler.get('raw', detector=detector, exposure=expId)
mData=raw.getMetadata()
raw_wcs = raw.getWcs()
calexp_wcs = calexp.getWcs()
print(calexp_wcs)

In [ ]:
import lsst.geom as geom
test = Angle(45*geom.degrees)
test.asDegrees()

In [ ]:
coord = SpherePoint(265.194977*geom.degrees, -18.456719*geom.degrees)
test_point = raw_wcs.skyToPixel(coord)

In [ ]:
from lsst.afw import cameraGeom
from lsst.obs.lsst.cameraTransforms import LsstCameraTransforms
from lsst.obs.lsst import LsstCam
camera = LsstCam.getCamera()
detName = 'R22_S11'

lct = LsstCameraTransforms(camera,detName)


In [ ]:
ccd_pixel = lct.focalMmToCcdPixel(-100.0, -200.0)
ccd_pixel

In [ ]:
#2025071200535
coord = SpherePoint(265.194977*geom.degrees, -18.456719*geom.degrees)
test_point = raw_wcs.skyToPixel(coord)
test_mm = lct.ccdPixelToFocalMm(test_point.x, test_point.y)
ccd_pixel = lct.focalMmToCcdPixel(test_mm.x, test_mm.y)
ccd_pixel

In [ ]:
#2025071200539
test_point = raw_wcs.skyToPixel(coord)
test_mm = lct.ccdPixelToFocalMm(test_point.x, test_point.y)
ccd_pixel = lct.focalMmToCcdPixel(test_mm.x, test_mm.y)
ccd_pixel

In [ ]:
#2025071200539
test_point = raw_wcs.skyToPixel(coord)
test_mm = lct.ccdPixelToFocalMm(test_point.x, test_point.y)
ccd_pixel = lct.focalMmToCcdPixel(test_mm.x, test_mm.y)
ccd_pixel

In [ ]:
for expId in range(517, 593):
    try:
        expId = 2025071200554
        detector = 94
        ref_calexp = butler.get('preliminary_visit_image', detector=detector, visit=expId)
        calexp_wcs = ref_calexp.getWcs()
        coord = SpherePoint(265.4*geom.degrees, -18.3*geom.degrees)
        test_point = calexp_wcs.skyToPixel(coord)
        test_mm = lct.ccdPixelToFocalMm(test_point.x, test_point.y)
        ccd_pixel = lct.focalMmToCcdPixel(test_mm.x, test_mm.y)
        print(expId, ":")
        print()
        print(ccd_pixel)
    except:
        continue


        

In [ ]:
expId = 2025071200586
detector = 30
ref_calexp = butler.get('preliminary_visit_image', detector=detector, visit=expId)
fig = plt.figure(figsize=(16,16))
arr = ref_calexp.image.array[0:600, 3000:3600]
myPlot = plot(arr, stretch='ccs', figure=fig)
ax = myPlot.get_axes()[0]
ax.scatter(493, 255\
            ,color='red', marker='x', s=500)

saveFilename = f"/home/c/cslage/u/LSSTCam/data/BlowUp3_{expId}_{detector}.png"
plt.savefig(saveFilename)